# Import libraries & read csv file

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/Train.csv')

# EDA

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
x_categories = ['Gender', 'Married', 'Education', 'Self_Employed', 'Credit_History', 'Property_Area']
for element in x_categories:
    cat_plot = pd.crosstab(df[element], df['Loan_Status'])
    cat_plot.plot(kind='bar', stacked=True, color=['red', 'green'])

In [ ]:
for category in x_categories:
    df[category] = df[category].astype('category')

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
numeric_categories = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
for element in numeric_categories:
    x = df[element]
    x.plot(kind='hist', bins = 30, fontsize = 20, figsize= (20,10), edgecolor='black', linewidth=1.2)
    plt.xlabel(element, fontsize=30)
    plt.ylabel('Frequency', fontsize=30)
    plt.show()

# Combining Applicant & CoapplicantIncome to one relevant feature

In [ ]:
total_income = df['ApplicantIncome'] + df['CoapplicantIncome']

# Log transforming total income & loan amount to normalize

In [ ]:
df['total_income_log'] = np.log(total_income)
df['LoanAmount_log'] = np.log(df['LoanAmount'])

df['total_income_log'].plot(kind='hist', bins = 30, fontsize = 20, figsize= (20,10), edgecolor='black', linewidth=1.2)
plt.xlabel(element, fontsize=30)
plt.ylabel('Frequency', fontsize=30)
plt.show()

df['LoanAmount_log'].plot(kind='hist', bins = 30, fontsize = 20, figsize= (20,10), edgecolor='black', linewidth=1.2)
plt.xlabel(element, fontsize=30)
plt.ylabel('Frequency', fontsize=30)
plt.show()

# Dropping useless columns

In [ ]:
df.drop(['Loan_ID', 'Loan_Amount_Term', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount'], axis=1, inplace=True)

# readying dependents to be more readable in (get dummies)

In [ ]:
df['Dependents'].replace('0', 'none', inplace=True)
df['Dependents'].replace('1', 'one', inplace=True)
df['Dependents'].replace('3+', 'three+', inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

# Converting data to dummies for ML algorithms to be able to process

In [ ]:
df_dummies = pd.get_dummies(df, columns = ['Gender', 'Dependents', 'Education', 'Self_Employed', 'Property_Area'])

In [ ]:
df_dummies.Married.replace(('Yes', 'No'), (1,0), inplace=True)
df_dummies.Loan_Status.replace(('Y', 'N'), (1,0), inplace=True)
df_dummies.head()

# Handling missing data

In [ ]:
df_dummies['total_income_log'].fillna(df_dummies['total_income_log'].mean(), inplace=True)
df_dummies['LoanAmount_log'].fillna(df_dummies['LoanAmount_log'].mean(), inplace=True)
df_dummies['Credit_History'].fillna(df_dummies['Credit_History'].value_counts().index[0], inplace=True)
df_dummies['Married'].fillna(df_dummies['Married'].value_counts().index[0], inplace=True)
df_dummies['Credit_History'] = df_dummies['Credit_History'].astype('float64')
df_dummies.info()

# Running ML Algorithms

- Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [ ]:
X = df_dummies.drop('Loan_Status', axis=1).values
y = df_dummies['Loan_Status'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space}
logreg = LogisticRegression()

logreg_cv = GridSearchCV(logreg, param_grid, cv=5)

In [ ]:
logreg_cv.fit(X_train, y_train)

In [ ]:
print("Tuned log reg parameter {}".format(logreg_cv.best_params_))
print("Highest score {}".format(logreg_cv.best_score_))